# IMPLEMENTING K_NEAREST_NEIGHBOUR 

### In the given data set we have to classify into which cluster an instance is going to fall


In [24]:
#Importing required predifined methods
import pandas as pd
import numpy as np
from collections import Counter
from math import sqrt
import random
import warnings
import operator
from scipy.io import arff

In [25]:
# reading the data into data frame with the name df
with open('trainProdSelection.arff') as f:
    data = arff.loadarff(f)
    Dtrain = pd.DataFrame(data[0])
Dtrain.head() # to print the top 5 values

,Type,LifeStyle,Vacation,eCredit,salary,property,label
0,b'student',b'spend>saving',6.0,40.0,13.62,3.2804,b'C1'
1,b'student',b'spend>saving',11.0,21.0,15.32,2.0232,b'C1'
2,b'student',b'spend>saving',7.0,64.0,16.55,3.1202,b'C1'
3,b'student',b'spend>saving',3.0,47.0,15.71,3.4022,b'C1'
4,b'student',b'spend>saving',15.0,10.0,16.96,2.2825,b'C1'


In [26]:
# reading the test data into a data frame with name dft
Dtrain.columns = ['Type', 'LifeStyle', 'Vacation', 'eCredit', 'Salary', 'Property', 'Label']
Dtrain.head() # to print the top 5 values


,Type,LifeStyle,Vacation,eCredit,Salary,Property,Label
0,b'student',b'spend>saving',6.0,40.0,13.62,3.2804,b'C1'
1,b'student',b'spend>saving',11.0,21.0,15.32,2.0232,b'C1'
2,b'student',b'spend>saving',7.0,64.0,16.55,3.1202,b'C1'
3,b'student',b'spend>saving',3.0,47.0,15.71,3.4022,b'C1'
4,b'student',b'spend>saving',15.0,10.0,16.96,2.2825,b'C1'


In [27]:
with open('testProdSelection.arff') as f:
    data = arff.loadarff(f)
    Dtest = pd.DataFrame(data[0])
Dtest.head() # to print the top 5 values

,Type,LifeStyle,Vacation,eCredit,salary,property,label
0,b'student',b'spend<saving',12.0,19.0,14.7900,3.7697,b'C1'
1,b'student',b'spend>>saving',29.0,10.0,16.1900,2.4839,b'C1'
2,b'student',b'spend<<saving',28.0,60.0,15.4600,1.1885,b'C1'
3,b'engineer',b'spend>saving',15.0,41.0,21.2600,1.4379,b'C1'
4,b'librarian',b'spend<saving',2.0,9.0,19.7207,0.6913,b'C1'


In [28]:
Dtest.columns = ['Type', 'LifeStyle', 'Vacation', 'eCredit', 'Salary', 'Property', 'Label']
Dtest.head() 

,Type,LifeStyle,Vacation,eCredit,Salary,Property,Label
0,b'student',b'spend<saving',12.0,19.0,14.7900,3.7697,b'C1'
1,b'student',b'spend>>saving',29.0,10.0,16.1900,2.4839,b'C1'
2,b'student',b'spend<<saving',28.0,60.0,15.4600,1.1885,b'C1'
3,b'engineer',b'spend>saving',15.0,41.0,21.2600,1.4379,b'C1'
4,b'librarian',b'spend<saving',2.0,9.0,19.7207,0.6913,b'C1'


In [29]:
#### Normalizing the values in the Vacation,eCredit,Salary,Property so that all the values range in between 0 to 1

#Storing the columns min and max, to normalize the testing data
min_max=[0.0]*8 
for attribute in list(Dtrain.columns.values):
    value = {}
    # Check the column can be normlized or not
    if attribute == 'eCredit' or attribute == 'Vacation' or attribute == 'Property' or attribute == 'Salary':
        value['maxi'] = max(Dtrain[attribute]) 
        value['mini'] = min(Dtrain[attribute])
        
        #storing the values in the array 
        if attribute == 'Vacation':
            min_max[0] = value['maxi']
            min_max[1] = value['mini']
        elif attribute == 'eCredit':
            min_max[2] = value['maxi']
            min_max[3] = value['mini']
        elif attribute == 'Salary':
            min_max[4] = value['maxi']
            min_max[5] = value['mini']
        elif attribute == 'Property':
            min_max[6] = value['maxi']
            min_max[7] = value['mini']
        
        #normalizing the values in the column
        Dtrain[attribute] = Dtrain[attribute].apply(lambda x: (x - value['mini'])/(value['maxi'] - value['mini']))

In [30]:
#### normalizing the testing data also with the help of values obtained in the train data set.
value = {}
value['maxi'] = 0
value['mini'] = 0
for attribute in list(Dtest.columns.values):
    
    # Check the column can be normlized or not
    if attribute == 'eCredit' or attribute == 'Vacation' or attribute == 'Property' or attribute == 'Salary':
        value['maxi'] = max(Dtest[attribute]) 
        value['mini'] = min(Dtest[attribute])
        
        #storing the values in the array 
        if attribute == 'Vacation':
            min_max[0] = value['maxi']
            min_max[1] = value['mini']
        elif attribute == 'eCredit':
            min_max[2] = value['maxi']
            min_max[3] = value['mini']
        elif attribute == 'Salary':
            min_max[4] = value['maxi']
            min_max[5] = value['mini']
        elif attribute == 'Property':
            min_max[6] = value['maxi']
            min_max[7] = value['mini']
        
        #normalizing the values in the column
        Dtest[attribute] = Dtest[attribute].apply(lambda x: (x - value['mini'])/(value['maxi'] - value['mini']))   

In [31]:

D=Dtrain
D.head()

,Type,LifeStyle,Vacation,eCredit,Salary,Property,Label
0,b'student',b'spend>saving',0.079365,0.107558,0.219960,0.183167,b'C1'
1,b'student',b'spend>saving',0.158730,0.052326,0.293102,0.112797,b'C1'
2,b'student',b'spend>saving',0.095238,0.177326,0.346023,0.174200,b'C1'
3,b'student',b'spend>saving',0.031746,0.127907,0.309882,0.189984,b'C1'
4,b'student',b'spend>saving',0.222222,0.020349,0.363663,0.127311,b'C1'


In [32]:
Dt=Dtest
Dt.head()

,Type,LifeStyle,Vacation,eCredit,Salary,Property,Label
0,b'student',b'spend<saving',0.20,0.058824,0.104637,0.398926,b'C1'
1,b'student',b'spend>>saving',0.54,0.021008,0.175059,0.243041,b'C1'
2,b'student',b'spend<<saving',0.52,0.231092,0.138339,0.085992,b'C1'
3,b'engineer',b'spend>saving',0.26,0.151261,0.430086,0.116229,b'C1'
4,b'librarian',b'spend<saving',0.00,0.016807,0.352657,0.025714,b'C1'


In [33]:
#converting both the data frame into list of lists
dataframe= Dtrain.values.tolist()
#print dataframe
dataframet= Dtest.values.tolist()


In [34]:
#### developing our own k nearest algorithm with respective to our requirments

def k_nearest(data_dic,predict,k):
    distances =[]
    for group in data_dic:
        i = 0
        for features in  data_dic[group]:
            euclidean_distance = 0
            if(predict[1] != features[1]):
                euclidean_distance += 1
            if(predict[0] != features[0]):
                euclidean_distance += 1
            x = predict[2]-features[2]
            y = predict[3]-features[3]
            z = predict[4]-features[4]
            w = predict[5]-features[5]
            euclidean_distance += (x**2 + y**2 + z**2 + w**2)
            if(sqrt(euclidean_distance) == 0):
                euclidean_distance = sqrt(euclidean_distance)
            else:
                euclidean_distance = 1/sqrt(euclidean_distance)
                
            distances.append([euclidean_distance,group])
    distances=sorted(distances)
    dictionary={b'C1':0.0,b'C2':0.0,b'C3':0.0,b'C4':0.0,b'C5':0.0}
    
    #we have to take the largest distances because we have used inverse eucildean
    lower = len(distances)-k
    higher = len(distances)
    for i in distances[lower:higher]:
        dictionary[i[1]] += i[0]
    sorted_dictionary = sorted(dictionary.items(), key=operator.itemgetter(1))
    return sorted_dictionary[4][0]
    
    
    
    
    
    #****PLEASE DONT CONSIDER THE CODE BELOW*****
#     votes = [i[1] for i in sorted(distances) [:k]]
#     #print distances
#     print votes
#     #print (Counter(votes).most_common(1))
#     vote_result = Counter(votes).most_common(1)[0][0]
#     #print vote_result
    
            
#     return vote_result


In [35]:
#creating a dicitionary to store instances without labels and Removing the cluster names from the instances and adding them to lists.
train_set={b'C1':[],b'C2':[],b'C3':[],b'C4':[],b'C5':[]} 
# keys are the cluster names
train_data= dataframe[:int(1*len(Dtrain))]
#assinging instances to particular key in the dictionary 
for i in train_data:
    train_set[i[-1]].append(i[:-1]) # removing the label and appending to specific list of the key


#removing labels for testing data also
final_test_set={b'C1':[],b'C2':[],b'C3':[],b'C4':[],b'C5':[]}
final_test_data=dataframet[:int(1*len(Dtest))]
for i in final_test_data:
    final_test_set[i[-1]].append(i[:-1])



#****PLEASE DONT CONSIDER THE CODE BELOW*****
#test_size=0.2
#test_set={'C1':[],'C2':[],'C3':[],'C4':[],'C5':[]}
#test_data= dataframe[-int(test_size*len(dataframe)):]

#assinging instances to particular key in the dictionary 



In [36]:
#sending our testing data to the model

#correct and total variables are used to calculate accuracy
correct =0.0
total =0.0

for group in final_test_set:
    for group_data in final_test_set[group]:
        vote = k_nearest(train_set,group_data,5)
        if group == vote: #if predicted cluster is correct 
            correct+=1
        total +=1

print ('Accuracy:', (correct)/total)

Accuracy: 0.2857142857142857
